In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)

input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
class Dinic:
  def __init__(self, n):
    self.lvl = [0] * n
    self.ptr = [0] * n
    self.q = [0] * n
    self.adj = [[] for _ in range(n)]

  def add_edge(self, a, b, c, rcap=0):
    self.adj[a].append([b, len(self.adj[b]), c, 0])
    self.adj[b].append([a, len(self.adj[a]) - 1, rcap, 0])

  def dfs(self, v, t, f):
    if v == t or not f:
      return f

    for i in range(self.ptr[v], len(self.adj[v])):
      e = self.adj[v][i]
      if self.lvl[e[0]] == self.lvl[v] + 1:
        p = self.dfs(e[0], t, min(f, e[2] - e[3]))
        if p:
          self.adj[v][i][3] += p
          self.adj[e[0]][e[1]][3] -= p
          return p
      self.ptr[v] += 1

    return 0

  def calc(self, s, t):
    flow, self.q[0] = 0, s
    for l in range(31):
      while True:
        self.lvl, self.ptr = [0] * len(self.q), [0] * len(self.q)
        qi, qe, self.lvl[s] = 0, 1, 1
        while qi < qe and not self.lvl[t]:
          v = self.q[qi]
          qi += 1
          for e in self.adj[v]:
            if not self.lvl[e[0]] and (e[2] - e[3]) >> (30 - l):
              self.q[qe] = e[0]
              qe += 1
              self.lvl[e[0]] = self.lvl[v] + 1

        p = self.dfs(s, t, INF)
        while p:
          flow += p
          p = self.dfs(s, t, INF)

        if not self.lvl[t]:
          break

    return flow

def sol() :
  N, P = map(int, input().split())
  C = Dinic(N)
  for _ in range(P) :
    a, b = map(lambda x : int(x) - 1, input().split())
    C.add_edge(a, b, 1)

  print(C.calc(0, 1))

def sol() :
  N, M, K = map(int, input().split())
  V = N + M + 3
  C = Dinic(V)

  C.add_edge(0, 1, K) #K명은 일을 추가로 할 수 있다.
  for i in range(N) :
    u = i + 2
    _, *l = map(int, input().split())
    C.add_edge(0, u, 1)
    C.add_edge(1, u, 1) #간선과 bridge를 연결해준다
    for j in l :
      v = N+j+1
      C.add_edge(u, v, 1)

  for i in range(M) : #각각의 일을 담당하는 사람은 1명이어야 한다
    C.add_edge(N+i+2, V-1, 1)

  print(C.calc(0, V-1))

sol()

### 자력솔
- 열혈강호 4를 먼저 풀긴 했는데 마찬가지로 bridge와의 추가 간선을 통해 추가로 일할 수 있는 사람의 처리를 했다.
- `K`명의 사람이 일을 2개할 수 있다 = 일을 1개씩 배치한 이후, `K`명의 사람은 일을 추가로 1개 더할 수 있다.
  - 따라서 sink와 bridge는 `K`만큼의 용량을 가지고, bridge와 직원들은 1의 용량을 가진다.